<a href="https://colab.research.google.com/github/RafaelCaballero/Julio24/blob/main/code/14correlaciones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introducción a la ciencia de datos con Python
### Rafa Caballero



# Correlaciones



### Índice
[Introducción](#Introducción)<br>
[Definición](#Definición)<br>
[Ejemplo](#Ejemplo)<br>
[A tener en cuenta](#Dibujar)<br>
[Representación gráfica](#Representación)<br>
[Variables nominales](#Nominales)<br>
[Bibliografía](#Bibliografía)<br>


<a name="Intro"></a>
## Introducción
Hasta ahora hemos considerado cada columna (cada variable) de forma independiente, viendo histogramas, outliers, etc. de forma individual. Solo hemos comparado columnas por ejemplo al hablar de los tests que nos permiten ver si tienen la misma media o si son muestras de la misma variable.

Ahora vamos a estudiar la *correlación* entre las columnas por pares. La idea es ver si dos variables tienen *relación lineal*: si cuando una crece la otra en general también (correlación positiva) o si cuando una crece la otra decrece (correlación negativa).

Una correlación alta indica que las variables están muy relacionadas (contienen información muy similar). Esto implica:

- Si una de las variables es una variable a predecir se podrá hacer muy bien a partir de la otra (por ejemplo con regresión lineal)

- Si las dos variables son características de un objeto puede que nos interese quitar una porque estamos "reforzando" o "duplicando" la misma información y eso puede tener influencia negativa en algunos métodos de aprendizaje automático.

Para más detalles ver la [bibliografía](#Bibliografía)

<a name="Definición"></a>
## Definición

Para estudiar la correlación entre dos variables se suele utilizar el *coeficiente de correlación de Pearson*

$$
r = \frac{\sum(x_i - \overline{x})(y_i - \overline{y})}{\sum(x_i - \overline{x})^2(y_i - \overline{y})^2}
$$

Es un valor que está entre -1 y +1. Algunos ejemplos:


<img src="https://www.statlect.com/images/linear-correlation-coefficient.png" />

<a name="Ejemplo"></a>
## Ejemplo
Por fortuna no tenemos que calcular las correlaciones; Pandas ya incluye un método [df.corr](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.corr.html)

Supongamos que queremos saber qué dos comunidades tienen mayor correlación en sus datos de paro

In [ ]:
import pandas as pd
url = "https://raw.githubusercontent.com/RafaelCaballero/tdm/master/datos/parocomunidades.csv"


df = pd.read_csv(url, encoding="latin1")
df["Comunidad"]  = 	df["Comunidad"].str.strip()
df

Vaya...las comunidades deberían ser columnas, qué hacer?

In [ ]:
df_ancho = df.pivot(index='Periodo', columns='Comunidad', values='Total')
df_ancho

In [ ]:
df_ancho.columns

In [ ]:
correlaciones = df_ancho.corr()
correlaciones

¿Entre qué dos comunidades diferentes hay mayor correlación? ¿y menos?

<a name="Dibujar"></a>
## A tener en cuenta

En el caso de las correlaciones (como en todos) no hay que precipitarse en sacar conclusiones. En particular:

- Que haya correlación no significa que los valores sean "parecidos" sino que se mueven a la vez (en la misma dirección o contraria)

- Correlación no implica causalidad, en ocasiones puede haber otra variable que esté "moviendo" las dos variables correlaciones

- Ya hemos visto que puede haber correlación de otros tipos que no está detectada por nuestro método (la correlación de Pearson); pero es que también sucede al revés, podemos encontrar correlación y que no signifique lo que creemos:

<img src="https://github.com/RafaelCaballero/tdm/raw/master/images/Situations-in-which-linear-correlation-should-not-be-used-a-two-variables-have-a.png" />

Vamos a mostrar los datos de las dos comunidades más correlacionadas a ver si es uno de estos casos:

In [ ]:
from matplotlib import pyplot as plt
fig, ax = plt.subplots(figsize=(10, 6))

ax.scatter(df_ancho["Comunitat Valenciana"],df_ancho['Murcia, Región de'])
ax.plot([0,30],[0,30],color="red")
plt.show()

In [ ]:
# otra forma
fig, ax = plt.subplots(figsize=(10, 6))
cs = ["Comunitat Valenciana",'Murcia, Región de']
for c in cs:
    df_ancho[c].plot(ax=ax,label=c)
ax.legend()
plt.show()

<a name="Representación"></a>
## Representación

La tabla de correlaciones se puede mostrar de forma más informativa, veamos cómo:

In [ ]:

correlaciones.style.background_gradient()

Con seaborn

In [ ]:
import seaborn as sns
plt.figure(figsize=(11,8))
sns.heatmap(correlaciones, cmap="Greens",annot=True)
plt.show()

Hay otras posibilidades más complejas que por ejemplo crean grupos (clústers) de columnas correlacionados

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

g = sns.clustermap(df_ancho.corr(),
                   method = 'complete',
                   cmap   = 'RdBu',
                   annot  = True,
                   annot_kws = {'size': 8})

Si se quieren ver las gráficas una a una (ojo, tarda mucho):

In [ ]:
sns.pairplot(df_ancho.corr())

<a name="Nominales"></a>
## Variables nominales

Si una variable es nominal/ordinal y la otra intervalo/ratio lo que se haría es convertir la variable nominal a números (ya veremos como) y usar correlación normal.

En el caso de dos variables "categóricas" (nominales u ordinales) podemos utilizar el test de la  $\mathcal{X}^2$ (chi-squared en inglés). Al ser un test solo dice si hay asociación o no, no cuánta (es mejor hablar de *asociación* dejando el término *correlación* para variables numéricas)

En este test $H_0$ = No hay asociación entre las variables.

Por tantp, si $p<0.05$ rechazaremos $H_0$ y diremos que hay asociación entre las variables con alta probabilidad, en otro caso no podemos asegurar que no exista asociación (ni que sí).

Ejemplo:

In [ ]:
import pandas as pd
url = "https://github.com/RafaelCaballero/tdm/raw/master/datos/Ventas%20Videojuegos.xlsx"
df = pd.read_excel(url)
df

In [ ]:
# las 5 plataformas más comunes
df2 = df[df.Plataforma.isin(df.Plataforma.value_counts()[0:5].index)]
df2

¿Están relacionados el género y la plataforma?

El primer paso es obtener la *tabla de contingencias* de las dos variables

In [ ]:
contingencias=pd.crosstab(index=df2.Plataforma,columns=df2.Genero)
contingencias

Es sobre esta tabla sobre la que hacemos el test:


In [ ]:
from scipy.stats import chi2_contingency

# este es el test
ChiSqResult = chi2_contingency(contingencias)
ChiSqResult

In [ ]:
p = ChiSqResult[1]
p

Como p es < 0.01 rechazamos la hipótesis nula; sí hay asociación entre las variables.

El método no nos dice qué tipo de relación, pero podemos verlo en a tabla de contingencias normalizada

In [ ]:
pd.crosstab(index=df2.Plataforma,columns=df2.Genero, normalize=0) # por filas

Otro ejemplo tomado de [aquí](https://thinkingneuron.com/how-to-measure-the-correlation-between-two-categorical-variables-in-python/)

In [ ]:

import pandas as pd
ColumnNames=['AGE','GENDER' ,'SALARY', 'APPROVE_LOAN']
DataValues=[ [28, 'M', 610000, 'Yes'],
             [42, 'M',140000, 'No'],
             [29, 'F',420000, 'No'],
             [30, 'M',420000, 'No'],
             [27, 'M',420000, 'No'],
             [34, 'F',190000, 'No'],
             [24, 'M',330000, 'Yes'],
             [34, 'M',160000, 'Yes'],
             [25, 'F',300000, 'Yes'],
             [34, 'M',450000, 'Yes'],
             [30, 'F',140000, 'Yes'],
             [33, 'M',600000, 'Yes'],
             [22, 'M',400000, 'Yes'],
             [25, 'F',490000, 'Yes'],
             [32, 'M',120000, 'Yes'],
             [29, 'F',360000, 'Yes'],
             [30, 'M',480000, 'Yes'],
             [29, 'F',460000, 'Yes'],
             [32, 'M',470000, 'Yes'],
             [28, 'M',400000, 'Yes']]
LoanData=pd.DataFrame(data=DataValues,columns=ColumnNames)
LoanData

¿Hay relación entre el sexo y que concedan el préstamo?

In [ ]:
from scipy.stats import chi2_contingency

contingencias=pd.crosstab(index=LoanData.GENDER,columns=LoanData.APPROVE_LOAN)
ChiSqResult = chi2_contingency(contingencias)
ChiSqResult[1]

Por tanto no podemos descartar que no haya asociación. Veamos un último caso:

In [ ]:
df2 = LoanData.copy()
df2["GENDER"] = LoanData.apply(lambda x: "M" if x["APPROVE_LOAN"]=="Yes" else "F",axis=1)
contingencias=pd.crosstab(index=df2.GENDER,columns=df2.APPROVE_LOAN)
ChiSqResult = chi2_contingency(contingencias)
ChiSqResult[1]

 La desventaja es que no vemos entre qué valores es la asociación, solo que existe.




<a name="Bibliografía"></a>
## Bibliografía


Una explicación matemática sencilla de donde he tomado alguna imagen:
*Taboga, Marco (2021). "Linear correlation", Lectures on probability theory and mathematical statistics. Kindle Direct Publishing. Online appendix.* https://www.statlect.com/fundamentals-of-probability/linear-correlation.



Sobre correlación y causalidad:
 [Why correlation does not imply causation?](https://medium.com/@seema.singh/why-correlation-does-not-imply-causation-5b99790df07e)



Descripción de los distintos tipos de tests para variables categóricas y numéricas:
*Data Preprocessing in Data Mining*. Intelligent Systems Reference Library №72. Salvador García, Julián Luengo, Francisco Herrera. Spring, 2015.  Ver en particular: *3.2.1 Finding Redundant Attributes*

